In [2]:
import numpy as np
import shap
import lightgbm as lgbm

# Modelo lightGBM para test
data, target = shap.datasets.boston(display=False)
data['MEDV']=target
y = 'MEDV'
X = [name for name in data.columns if name not in [y]]
split_ratio = 0.80
split = np.random.rand(len(data)) < split_ratio
train, test = data[split], data[~split]
train_dataframe =lgbm.Dataset(data=train[X],label=train[y].values,free_raw_data=False)
valid_dataframe =lgbm.Dataset(data =test[X],label=test[y].values,free_raw_data=False)
model_params = {"objective": "regression","metric": "rmse","boosting": "gbdt","learning_rate": 0.01,"num_leaves": 31,
                "max_depth": -1,"min_data_in_leaf": 20,"bagging_fraction": 0.9,"feature_fraction": 0.7,"lambda_l2": 0.,
                "seed": 42,"deterministic": True,"verbose": -1}
model = lgbm.train(train_set = train_dataframe,valid_sets = [valid_dataframe, ],num_boost_round = 1000,
                   early_stopping_rounds = 50,params = model_params,verbose_eval=False)
test_yhat = test.copy()
yhat, resid = 'prediction', 'residuals'
test_yhat[yhat] = model.predict(test[X])
test_yhat[resid] = test_yhat[y] - test_yhat[yhat]

In [4]:
from calculate import *

In [16]:
shap_values = get_shap_values(model, test[X])

In [12]:
str(type(shap_values))=="<class 'shap._explanation.Explanation'>"

True

In [24]:
assert type(shap_importance(X, shap_values))==tuple
assert type(shap_importance(X, shap_values)[0])==list
assert type(shap_importance(X, shap_values)[1])==np.ndarray

In [28]:
type(permutation_VI(model, test[X])[1])

list